In [11]:
import os
import shutil
from PIL import Image
import yaml

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

def list_imgs(dirpath):
    # Accept .jpg / .jpeg (case-insensitive)
    exts = {".jpg", ".JPG"}
    files = [f for f in os.listdir(dirpath) if os.path.splitext(f)[1].lower() in exts]
    files.sort()
    return files

# ---------- load config ----------
with open("../config/config.yml", "r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)

o_positive_path = cfg["paths"]["o_positive_path"]
positive_path   = cfg["paths"]["positive_path"]

o_neg_img_path  = cfg["paths"]["o_negative_path"] + "/images"
o_neg_lbl_path  = cfg["paths"]["o_negative_path"] + "/labels"
neg_out_root    = cfg["paths"]["negative_path"]

neg_out_img = ensure_dir(os.path.join(neg_out_root, "images"))
neg_out_lbl = ensure_dir(os.path.join(neg_out_root, "labels"))

# ---------- positives: rename to 0.JPG, 1.JPG, ... ----------
ensure_dir(positive_path)
pos_files = list_imgs(o_positive_path)

for idx, filename in enumerate(pos_files):
    src = os.path.join(o_positive_path, filename)
    with Image.open(src) as img:
        dst = os.path.join(positive_path, f"{idx}.JPG")
        img.save(dst)
print(f"Positives done: {len(pos_files)} files -> {positive_path}")

# ---------- negatives: images + matching labels ----------
neg_img_files = list_imgs(o_neg_img_path)

missing_labels = []
for idx, filename in enumerate(neg_img_files):
    stem, _ = os.path.splitext(filename)
    img_src = os.path.join(o_neg_img_path, filename)
    lbl_src = os.path.join(o_neg_lbl_path, f"{stem}.txt")

    # save image as idx.JPG
    with Image.open(img_src) as img:
        img_dst = os.path.join(neg_out_img, f"{idx}.JPG")
        img.save(img_dst)

    # copy label as idx.txt (if exists)
    if os.path.exists(lbl_src):
        lbl_dst = os.path.join(neg_out_lbl, f"{idx}.txt")
        shutil.copyfile(lbl_src, lbl_dst)
    else:
        missing_labels.append(stem)

print(f"Negatives done: {len(neg_img_files)} images -> {neg_out_img}")
print(f"              labels -> {neg_out_lbl}")
if missing_labels:
    print(f"Warning: {len(missing_labels)} image(s) had no matching .txt label:")
    for s in missing_labels[:10]:
        print(f"  - {s}.txt")
    if len(missing_labels) > 10:
        print("  ...")


Positives done: 1324 files -> E:/data/Aging_walls/processed/positive
Negatives done: 1303 images -> E:/data/Aging_walls/processed/negative\images
              labels -> E:/data/Aging_walls/processed/negative\labels
